# **[Jigsaw Rate Severity of Toxic Comments](https://www.kaggle.com/c/jigsaw-toxic-severity-rating/)**
_11/08/2021 ~ 02/07/2022_  
kaggleのJigsawコンペに参加したので振り返る。
***

### **概要**
各コメントに対して、相対的な毒性の強さを表すスコア(≒有害度)を予測する。

#### 手順
1. 各種ライブラリのインポート
2. データの可視化
3. 考察とモデル決め
4. データの加工と、予測
5. 提出用ファイルの作成

### **1. 各種ライブラリのインポート**

In [21]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge

### **2. データの可視化**

In [22]:
comments = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
comments.head()

,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...


今回は上記の _text_ 列の各コメントに、毒性の強さを表すスコアを付すことが目標となる。

In [23]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7537 entries, 0 to 7536
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   comment_id  7537 non-null   int64 
 1   text        7537 non-null   object
dtypes: int64(1), object(1)
memory usage: 117.9+ KB


7537行からなるデータの全てにおいて欠損値は無く、各コメントには _comment_id_ 列で定められるIDが付されている。

In [24]:
prev_train = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
prev_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


学習用のデータ。  
_toxic, severe_toxic, obscene, threat, insult, identity_hate_ 列で、各コメントの毒性の種類が分類分けされている。

In [25]:
types= ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
print(prev_train.shape)
for type in types:
    print(type, prev_train[type].sum())

(159571, 8)
toxic 15294
severe_toxic 1595
obscene 8449
threat 478
insult 7877
identity_hate 1405


159,571行にわたる学習用データのコメントの中で、_toxic_ であるものは15,294と10％程度を占める。  
_threat_ であるものにあたっては478と、僅か0.3%。  
毒性の種類によって出現頻度が異なるのに関わらず、カウントが全て変わらず1であるというのは、毒性の種類差に伴うその毒性の強度差を適切に示さないと考えたため、カウントを出現頻度に応じて重み付けをするという発想になる。

In [26]:
validation = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
validation.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


このデータでは、2つのコメントを与えられたアノテーターがどちらをより毒性が強いと判断したかが示されている。  
モデル構築後、この両者のコメントそれぞれに予測されたスコアを比較し、大小関係を調べることでそのモデルの精度をある程度確かめることができると推測できる。

### **3. 考察とモデル決め**

私はコメントの毒性を考えるにあたって、コメント内に _toxic_ な単語が含まれるからこそ、そのコメントが _toxic_ たり得ると考えた。  
コメント内におけるそのような単語の存在をどうにか数値化出来ないかと調べたところ、TF-IDFというものを見つけた。

>tf-idfとは、各文書（document）中に含まれる各単語（term）が「その文書内でどれくらい重要か」を表す統計的尺度の一つで、具体的には「ある文書内」で「ある単語」が「どれくらい多い頻度で出現するか」を表すtf（term frequency：単語頻度）値と、「全文書中」で「ある単語を含む文書」が「（逆に）どれくらい少ない頻度で存在するか」を表すidf（inverse document frequency：逆文書頻度）値を掛け合わせた値のことである。  
出典：[tf-idf（term frequency - inverse document frequency）とは？](https://atmarkit.itmedia.co.jp/ait/articles/2112/23/news028.html)  
  
各コメントをこの尺度に従いベクトル化すれば、それらの数値を特徴量とし、上記で重み付けをした各カウントの合計を目的変数とするような相関関係でモデルを構築できると考えた。
またこのコンペでは、Leaderboard上で算出されるスコアがテストデータの5％しか用いられていないことから、過学習を防ぐためにRidge回帰を用いる手法が人気であったため、それを採択した。

### **4. データの加工と予測**

In [27]:
rates = {}
for type in types:
    rates.update({"{}".format(type): prev_train[type].sum() / prev_train["id"].count()})
rates = pd.Series(rates.values(), index=rates.keys())
rates #出現率を示す。反省点１

toxic            0.095844
severe_toxic     0.009996
obscene          0.052948
threat           0.002996
insult           0.049364
identity_hate    0.008805
dtype: float64

In [28]:
for type in types:
    prev_train[type] = prev_train[type] / rates[type]
prev_train["total"] = prev_train[types].sum(axis=1).astype(np.float64)
prev_train[prev_train["total"] > 0].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,total
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,10.433569,100.044514,18.886377,0.0,20.257839,0.000000,149.622299
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,10.433569,0.000000,0.000000,0.0,0.000000,0.000000,10.433569
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",10.433569,0.000000,0.000000,0.0,0.000000,0.000000,10.433569
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,10.433569,0.000000,18.886377,0.0,20.257839,113.573665,163.151451
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",10.433569,0.000000,18.886377,0.0,20.257839,0.000000,49.577785


In [29]:
train = prev_train[["comment_text", "total"]].copy()
train.head() #データが正しく加工できているかの確認

,comment_text,total
0,Explanation\nWhy the edits made under my usern...,0.0
1,D'aww! He matches this background colour I'm s...,0.0
2,"Hey man, I'm really not trying to edit war. It...",0.0
3,"""\nMore\nI can't make any real suggestions on ...",0.0
4,"You, sir, are my hero. Any chance you remember...",0.0


In [30]:
vectorizer = TfidfVectorizer(analyzer= "char_wb", ngram_range=(3,5), min_df=3, max_df=0.5)
tfidf = vectorizer.fit_transform(train["comment_text"])
tfidf_comment = vectorizer.transform(comments["text"])

In [31]:
alphas = [0.3, 0.4, 0.5, 0.6, 0.7]
#Ridge回帰では、αの値によりモデルの複雑さを制御できる。(=学習の度合いを制御できる。)
#適切なαを探す方法が分からずじまいだったので、いくつかの数値で求められた答えの平均をとることにした。反省点２

In [32]:
X_less = vectorizer.transform(validation["less_toxic"])
X_more = vectorizer.transform(validation["more_toxic"])

In [33]:
model0 = Ridge(alpha=alphas[0])
model0.fit(tfidf, train["total"])
pred0 = model0.predict(tfidf_comment)
pred_less = model0.predict(X_less)
pred_more = model0.predict(X_more)
(pred_less < pred_more).mean()

0.672047296399628

In [34]:
model1 = Ridge(alpha=alphas[1])
model1.fit(tfidf, train["total"])
pred1 = model1.predict(tfidf_comment)
pred_less = model1.predict(X_less)
pred_more = model1.predict(X_more)
(pred_less < pred_more).mean()

0.6722465789823303

In [35]:
model2 = Ridge(alpha=alphas[2])
model2.fit(tfidf, train["total"])
pred2 = model2.predict(tfidf_comment)
pred_less = model2.predict(X_less)
pred_more = model2.predict(X_more)
(pred_less < pred_more).mean()

0.6726119303839511

In [36]:
model3 = Ridge(alpha=alphas[3])
model3.fit(tfidf, train["total"])
pred3 = model3.predict(tfidf_comment)
pred_less = model3.predict(X_less)
pred_more = model3.predict(X_more)
(pred_less < pred_more).mean()

0.6725787166201674

In [37]:
model4 = Ridge(alpha=alphas[4])
model4.fit(tfidf, train["total"])
pred4 = model4.predict(tfidf_comment)
pred_less = model4.predict(X_less)
pred_more = model4.predict(X_more)
(pred_less < pred_more).mean()

0.6729772817855719

_α_ を0.1程度変化させるようでは、精度は0.0数%しか変わらないようだ。

### **5. 提出用ファイルの作成**

In [38]:
result = pd.DataFrame()
result["comment_id"] = comments["comment_id"]
result["score"] = (pred0 + pred1 + pred2 + pred3 + pred4) / 5
result.to_csv("submission.csv", index=False)
print(result.shape)
result.head()

(7537, 2)


,comment_id,score
0,114890,-3.437474
1,732895,9.442339
2,1139051,12.974066
3,1434512,1.374056
4,2084821,4.679146


### **振り返り**
結果的にこのコンペでは、PublicLB(=Leaderboard)でのスコア(=成績)と、PrivateLBでのスコアが大きく異なるものとなった。PublicLBではテストデータの5％のみしか使用されず、過学習していてもPublicLBでのスコアが高くなる一方で、それに気づきにくく誤解を生みやすかったことが要因のようだ。  
1000位も順位が変わる人がいる一方、私の順位の変化は10位下がっただけで、その順位変化は周りに比べ小さかったといえる。これはRidge回帰という過学習を防ぎやすいモデルを選択したことに加え、 _α_ の値を小さくしたことでより過学習を防げたからであると考える。他者のモデルをみると、考察通り過学習するようなモデルは大きく順位が下がっていることからもこれは正しいだろう。  
  
反省点としては大きく2点ある。学習用データの目的変数における加工に関してが1点、Ridge回帰におけるバラメータ _α_ の設定に関してがもう1点である。  
1点目に関して、カテゴリ分類におけるカウントを重み付けすることはよかったと考える一方、そのコメントの評価値を重み付けしたカウントの和にするのが誤りであった。毒性のカテゴリ分類において、多くのカテゴリに属することとそのコメントの毒性が強いということは必ずしも一致する訳ではないからだ。パラメーターチューニングのような形で、より適切な評価値を算出できたのではないかと考える。  
もう一点に関して、これは交差検証により解決できると考える。RidgeCVというモデルでは、上記のRidge回帰に加え、精度が高くなるように _α_ の値をチューニングしてくれるようだ。これにより、より適切な _α_ を用いて予測結果を算出することができたと考える。  

上位のモデルでは、BERTというモデルが使われているようだ。これはDeepL等でも使われているモデルで、自然言語処理に強いモデルのようである。また自然言語処理コンペに出るとなった時には、このモデルを使ってみたい。